### Effet du support

Ce graphique interactif illustre l’effet du changement de support sur la distribution des teneurs.

Au départ, deux gisements sont simulés avec la **même distribution statistique** à l’échelle du **support ponctuel (1 m × 1 m)**.  
Pour observer l’impact de l’agrandissement du support, vous pouvez utiliser le **widget interactif ci-dessous** afin de modifier la taille du bloc de support.

Par exemple, la teneur d’un bloc de **10 m × 10 m** est calculée comme la **moyenne des teneurs** des cellules de **1 m × 1 m** qu’il contient.

Les **cartes**, les **histogrammes** et les **fonctions de répartition cumulées** sont mis à jour dynamiquement à chaque changement de support. Cela vous permet de visualiser concrètement l’**effet de l’agrégation spatiale** sur les propriétés statistiques du gisement.

**Amusez-vous à explorer les résultats — nous en discuterons en classe !**



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider
from numpy.fft import fft2, ifft2, fftshift

# --- Covariance sphérique (FFT)
def spherical_covariance_fft(size, range_, sill=1.0):
    extended_size = 2 * size
    x = np.arange(-extended_size//2, extended_size//2)
    X, Y = np.meshgrid(x, x)
    h = np.sqrt(X**2 + Y**2)
    h = np.minimum(h, range_ * size)
    cov = sill * (1 - 1.5 * (h / (range_ * size)) + 0.5 * (h / (range_ * size))**3)
    cov[h > (range_ * size)] = 0
    return fftshift(cov)

# --- FFT-MA avec recadrage à N×N
def fftma_simulation(size, range_, sill=1.0, seed=0):
    np.random.seed(seed)
    extended_size = 2 * size
    cov_model = spherical_covariance_fft(size, range_, sill)
    cov_fft = fft2(cov_model)
    white_noise = np.random.normal(size=(extended_size, extended_size))
    white_fft = fft2(white_noise)
    z_fft = np.sqrt(np.abs(cov_fft)) * white_fft
    z_ext = np.real(ifft2(z_fft))
    start = extended_size // 4
    end = start + size
    return z_ext[start:end, start:end]

# --- Transformation lognormale
def gaussian_to_lognormal(field):
    return np.exp(field)

# --- Apparier les histogrammes
def match_histogram(reference, target):
    flat_ref = np.sort(reference.ravel())
    sorted_idx = np.argsort(target.ravel())
    result = np.zeros_like(target.ravel())
    result[sorted_idx] = flat_ref
    return result.reshape(target.shape)

# --- Agrégation (effet de support)
def aggregate(field, block_size):
    s = field.shape[0]
    reduced_size = s // block_size
    aggregated = np.zeros((reduced_size, reduced_size))
    for i in range(reduced_size):
        for j in range(reduced_size):
            block = field[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
            aggregated[i, j] = np.mean(block)
    return aggregated

# --- Simulation initiale
size = 500
range_short = 10/size
range_long = 250/size

gauss_short = fftma_simulation(size=size, range_=range_short, seed=42)
gauss_long = fftma_simulation(size=size, range_=range_long, seed=24)

lognorm_short = gaussian_to_lognormal(gauss_short)
lognorm_long = gaussian_to_lognormal(gauss_long)
lognorm_long = match_histogram(lognorm_short, lognorm_long)

# --- Affichage interactif avec échelle fixe
def plot_fields(support):
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    agg_short = aggregate(lognorm_short, support)
    agg_long = aggregate(lognorm_long, support)

    vmin, vmax = 0, 10
    bins = np.linspace(0, 10, 11)

    # Moyenne unique utilisée pour les deux visualisations
    mean = np.mean(agg_short)

    # --- Carte portée courte ---
    im0 = axes[0, 0].imshow(agg_short, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[0, 0].set_title(f'Portée courte – Support {support}×{support}', fontsize=12)
    axes[0, 0].axis('off')
    fig.colorbar(im0, ax=axes[0, 0], fraction=0.046, pad=0.04)

    # --- Carte portée longue ---
    im1 = axes[0, 1].imshow(agg_long, cmap='viridis', vmin=vmin, vmax=vmax)
    axes[0, 1].set_title(f'Portée longue – Support {support}×{support}', fontsize=12)
    axes[0, 1].axis('off')
    fig.colorbar(im1, ax=axes[0, 1], fraction=0.046, pad=0.04)

    # --- Histogrammes superposés ---
    axes[1, 0].hist(agg_short.ravel(), bins=bins, alpha=0.6, color='steelblue', label='Portée courte', edgecolor='black')
    axes[1, 0].hist(agg_long.ravel(), bins=bins, alpha=0.6, color='darkorange', label='Portée longue', edgecolor='black')
    axes[1, 0].axvline(mean, color='black', linestyle='--', linewidth=2, label=f'Moyenne = {mean:.2f}')
    axes[1, 0].set_title('Histogrammes superposés', fontsize=12)
    axes[1, 0].set_xlabel('Teneur')
    axes[1, 0].set_ylabel('Fréquence')
    axes[1, 0].set_xlim([0, 10])
    axes[1, 0].grid(True)
    axes[1, 0].legend()

    # --- ECDF superposées ---
    for data, color, label in zip(
        [agg_short.ravel(), agg_long.ravel()],
        ['steelblue', 'darkorange'],
        ['Portée courte', 'Portée longue']
    ):
        sorted_data = np.sort(data)
        ecdf = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
        axes[1, 1].plot(sorted_data, ecdf, color=color, lw=2, label=label)

    # Lignes de moyennes en noir
    axes[1, 1].axvline(mean, color='black', linestyle='--', linewidth=2, label=f'Moyenne = {mean:.2f}')

    axes[1, 1].set_title('Fonctions de répartition cumulées', fontsize=12)
    axes[1, 1].set_xlabel('Teneur')
    axes[1, 1].set_ylabel('F(x)')
    axes[1, 1].set_xlim([0, 10])
    axes[1, 1].set_ylim([0, 1])
    axes[1, 1].grid(True)
    axes[1, 1].legend()

    plt.tight_layout()
    plt.show()


# --- Widget interactif
interact(plot_fields, support=IntSlider(min=1, max=50, step=2, value=1, description='Support (m)'))